In [1]:
from pyspark.sql.types import *
from pyspark.sql.window import Window

import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

25/05/14 02:24:48 WARN Utils: Your hostname, codespaces-367222 resolves to a loopback address: 127.0.0.1; using 10.0.5.131 instead (on interface eth0)
25/05/14 02:24:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/14 02:24:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/14 02:24:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/05/14 02:24:49 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
spark

In [13]:
spark.read.text("hdfs://172.27.1.5:8020/user/datapath/datasets/products").show()

+--------------------+
|               value|
+--------------------+
|45,,1009,http://i...|
|46,,1010,http://i...|
|46,,1011,http://i...|
|46,,1012,http://i...|
|46,,1013,http://i...|
|46,,1014,http://i...|
|46,,1015,http://i...|
|46,,1016,http://i...|
|46,,1017,http://i...|
|46,,1018,http://i...|
|46,,1019,http://i...|
|46,,1020,http://i...|
|46,,1021,http://i...|
|46,,1022,http://i...|
|46,,1023,http://i...|
|46,,1024,http://i...|
|46,,1025,http://i...|
|46,,1026,http://i...|
|46,,1027,http://i...|
|46,,1028,http://i...|
+--------------------+
only showing top 20 rows



In [14]:
#CUSTOMERS_DATA =   'hdfs://172.27.1.5:8020/user/datapath/datasets/customers'
#DEPARTMENTS_DATA = 'hdfs://172.27.1.5:8020/user/datapath/datasets/departments'
#CATEGORIES_DATA =  'hdfs://172.27.1.5:8020/user/datapath/datasets/categories'
PRODUCTS_DATA =    'hdfs://172.27.1.5:8020/user/datapath/datasets/products'
#ORDERS_DATA =      'hdfs://172.27.1.5:8020/user/datapath/datasets/orders'
#ORDER_ITEMS_DATA = 'hdfs://172.27.1.5:8020/user/datapath/datasets/order_items'

In [8]:
# define the schema, corresponding to a line in the csv data file for Customer
customers_schema = StructType([
    StructField('customer_id',       IntegerType(), nullable=True),
    StructField('customer_fname',    StringType(), nullable=True),
    StructField('customer_lname',    StringType(), nullable=True),
    StructField('customer_email',    StringType(), nullable=True),
    StructField('customer_password', StringType(), nullable=True),
    StructField('customer_street',   StringType(), nullable=True),
    StructField('customer_city',     StringType(), nullable=True),
    StructField('customer_state',    StringType(), nullable=True),
    StructField('customer_zipcode',  StringType(), nullable=True)])

In [ ]:
departments_schema = StructType([
    StructField('department_id',   IntegerType(), nullable=True),
    StructField('department_name', StringType(), nullable=True)])

In [ ]:
categories_schema = StructType([
    StructField('category_id',            IntegerType(), nullable=True),
    StructField('category_department_id', IntegerType(), nullable=True),
    StructField('category_name',          StringType(), nullable=True)])

In [15]:
products_schema = StructType([
    StructField('product_id',          IntegerType(), nullable=True),
    StructField('product_category_id', IntegerType(), nullable=True),
    StructField('product_name',        StringType(), nullable=True),
    StructField('product_description', StringType(), nullable=True),
    StructField('product_price',       FloatType(), nullable=True),
    StructField('product_image',       StringType(), nullable=True)])

In [ ]:
orders_schema = StructType([
    StructField('order_id',          IntegerType(), nullable=True),
    StructField('order_date',        StringType(), nullable=True),
    StructField('order_customer_id', IntegerType(), nullable=True),
    StructField('order_status',      StringType(), nullable=True)])

In [ ]:
order_items_schema = StructType([
    StructField('order_item_id',            IntegerType(), nullable=True),
    StructField('order_item_order_id',      IntegerType(), nullable=True),
    StructField('order_item_product_id',    IntegerType(), nullable=True),
    StructField('order_item_quantity',      IntegerType(), nullable=True),
    StructField('order_item_subtotal',      FloatType(), nullable=True),
    StructField('order_item_product_price', FloatType(), nullable=True)])

### Cargamos los datos

In [16]:

#customers_df = spark.read.option("header", "true").csv(path=CUSTOMERS_DATA, schema=customers_schema)
#customers_df.cache()

#departments_df = spark.read.option("header", "true").csv(path=DEPARTMENTS_DATA, schema=departments_schema)
#departments_df.cache()

#categories_df = spark.read.option("header", "true").csv(path=CATEGORIES_DATA, schema=categories_schema)
#categories_df.cache()

products_df = spark.read.option("header", "true").csv(path=PRODUCTS_DATA, schema=products_schema)
products_df.cache()

#orders_df = spark.read.option("header", "true").csv(path=ORDERS_DATA, schema=orders_schema)
#orders_df.cache()

#order_items_df = spark.read.option("header", "true").csv(path=ORDER_ITEMS_DATA, schema=order_items_schema)
#order_items_df.cache()

DataFrame[product_id: int, product_category_id: int, product_name: string, product_description: string, product_price: float, product_image: string]

In [17]:
#customers_df.show()
products_df.show()

25/05/14 02:43:06 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: 31, , 673, http://images.acmesports.sports/PING+G30+Fairway+Wood, PING G30 Fairway Wood, 249.0
 Schema: product_id, product_category_id, product_name, product_description, product_price, product_image
Expected: product_id but found: 31
CSV file: hdfs://172.27.1.5:8020/user/datapath/datasets/products/part-m-00002
25/05/14 02:43:06 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: 45, , 1009, http://images.acmesports.sports/Diamond+Fear+No+Evil+Compound+Bow+Package, Diamond Fear No Evil Compound Bow Package, 599.99
 Schema: product_id, product_category_id, product_name, product_description, product_price, product_image
Expected: product_id but found: 45
CSV file: hdfs://172.27.1.5:8020/user/datapath/datasets/products/part-m-00003
25/05/14 02:43:06 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: 16, , 337, http://images.acmesports.sports/Fitness+

+----------+-------------------+------------+--------------------+-------------+-------------+
|product_id|product_category_id|product_name| product_description|product_price|product_image|
+----------+-------------------+------------+--------------------+-------------+-------------+
|        46|               NULL|        1010|http://images.acm...|         NULL|        19.98|
|        46|               NULL|        1011|http://images.acm...|         NULL|       499.99|
|        46|               NULL|        1012|http://images.acm...|         NULL|       299.99|
|        46|               NULL|        1013|http://images.acm...|         NULL|       349.99|
|        46|               NULL|        1014|http://images.acm...|         NULL|        49.98|
|        46|               NULL|        1015|http://images.acm...|         NULL|       399.99|
|        46|               NULL|        1016|http://images.acm...|         NULL|       549.99|
|        46|               NULL|        1017|http:

### Creamos vistas temporales para trabajar con spark.sql

In [12]:
customers_df.createOrReplaceTempView("customers")
customers_df.show(5)

+-----------+--------------+--------------+--------------+-----------------+---------------+-------------+--------------------+----------------+
|customer_id|customer_fname|customer_lname|customer_email|customer_password|customer_street|customer_city|      customer_state|customer_zipcode|
+-----------+--------------+--------------+--------------+-----------------+---------------+-------------+--------------------+----------------+
|       NULL|     XXXXXXXXX|          Mary|          9328|            Perez|      XXXXXXXXX|           CA|  376 Golden Orchard|           92553|
|       NULL|     XXXXXXXXX|        Eugene|          9329|           Powell|      XXXXXXXXX|           LA|   2161 Burning Maze|           70003|
|       NULL|     XXXXXXXXX|          Mary|          9330|           Conley|      XXXXXXXXX|           PR| 3046 Broad Sky Dale|           00725|
|       NULL|     XXXXXXXXX|         Donna|          9331|            Smith|      XXXXXXXXX|           NJ|941 Thunder Branc...|   

In [ ]:
departments_df.createOrReplaceTempView("departments")
departments_df.show(5)

In [ ]:
orders_df.createOrReplaceTempView("orders")
orders_df.show(5)

In [ ]:
order_items_df.createOrReplaceTempView("order_items")
order_items_df.show(5)

In [ ]:
products_df.createOrReplaceTempView("products")
products_df.show(5)

In [ ]:
categories_df.createOrReplaceTempView("categories")
categories_df.show(5)